In [1]:
import torch
from copy import copy
from transformers import AutoTokenizer
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import os
from math import inf
from datasets import load_dataset,concatenate_datasets
from transformers import AutoModelForMultipleChoice
from adapters import init,AutoAdapterModel
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer
import pandas as pd
from adapters import AdapterSetup, AutoAdapterModel
from evaluate import load
from transformers import DataCollatorForSeq2Seq,RobertaForMultipleChoice,RobertaTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from nltk.tokenize import sent_tokenize
import nltk

nltk.download("punkt")


2024-05-02 15:57:36.955261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 15:57:36.955468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 15:57:37.038010: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 15:57:37.202850: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 15:57:38.713838: W tensorflow/compiler/tf2

True

In [2]:
def preprocess_function(examples):
    max_input_length = 1024
    max_target_length = 128
    prefix = "summarize: "    
    inputs = [prefix + doc for doc in examples["source"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["target"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

  
def compute_metrics_dia_wikilingua(eval_preds):
        print('hi')
        print("eval_preds",eval_preds)
        predictions, labels = eval_preds

    
        # print(predictions)
        # print(labels)
    
        if isinstance(predictions, tuple):
            predictions = predictions[0]
            

        print(type(predictions))
        # print(predictions.size())
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        # Rouge expects a newline after each sentence
        decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
        
        # Note that other metrics may not have a `use_aggregator` parameter
        # and thus will return a list, computing a metric for each sentence.
        result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
        # Extract a few results
        result = {key: value * 100 for key, value in result.items()}
        
        # Add mean generated length
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)
        
        return {k: round(v, 4) for k, v in result.items()}

In [3]:
new_dialects = ["std-dia", "aus", "hon", "nig", "col", "wel"]

train_dialects =["std-dia", "aus", "hon", "nig", "col", "wel"]
test_dialects =["std-dia", "aus", "hon", "nig", "col", "wel",'chi','iri','pak']

# train_dialects= [ "aus"]



In [4]:
metric = load("rouge")

model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
init(model) 
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
########33 Loading DS1

DS = {}

for dialect in new_dialects:

    if dialect != "std-dia":
        ds = load_dataset("ashabrawy/dia_wikilingua", dialect).remove_columns(['Unnamed: 0'])
        # columns = copy(ds['train'].column_names)
        # # columns.remove('label')
        # columns.remove('labels')
        ds = ds.map(preprocess_function, batched=True)


    else:
        ds = load_dataset('GEM/wiki_lingua')

        ## Here the dataset reference has type list, so we need to select the first element, to be able to concat it with dialects dataset 
        ds['train'] = ds['train'].select(range(500)).map(lambda example: {"references": example["references"][0]})
        ds['validation'] = ds['validation'].select(range(500)).map(lambda example: {"references": example["references"][0]})
        ds = ds.map(preprocess_function, batched=True)
        

    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    DS[dialect] = ds

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for GEM/wiki_lingua contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/GEM/wiki_lingua
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/27489 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:

for dialect in ['iri','chi','pak']:
    ds = load_dataset("ashabrawy/dia_wikilingua", dialect)
    ds = ds.map(preprocess_function, batched=True)
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    DS[dialect] = ds

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
# DS.items()

In [9]:
for k,d in DS.items():

    # if ['gem_id', 'gem_parent_id'] in DS[k]['train'].column_names:
    #     DS[k]['train'] = DS[k]['train'].remove_columns(['gem_id', 'gem_parent_id'])
    # if ['gem_id', 'gem_parent_id'] in DS[k]['validation'].column_names:
    
    #     DS[k][''] = DS[k]['validation'].remove_columns(['gem_id', 'gem_parent_id'])


    if k in ['iri','chi','pak']:
        DS[k]['validation'] = DS[k]['test'].select_columns(['input_ids', 'attention_mask', 'labels'])
    else:
        DS[k]['validation'] = DS[k]['validation'].select_columns(['input_ids', 'attention_mask', 'labels'])
    
    
    

In [10]:
for k,d in DS.items():
    print( ['gem_id', 'gem_parent_id'] in  DS[k]['train'].column_names)
    break

False


In [11]:
DS

{'std-dia': DatasetDict({
     train: Dataset({
         features: ['gem_id', 'gem_parent_id', 'source_language', 'target_language', 'source', 'target', 'references', 'input_ids', 'attention_mask', 'labels'],
         num_rows: 500
     })
     validation: Dataset({
         features: ['input_ids', 'attention_mask', 'labels'],
         num_rows: 500
     })
     test: Dataset({
         features: ['gem_id', 'gem_parent_id', 'source_language', 'target_language', 'source', 'target', 'references', 'input_ids', 'attention_mask', 'labels'],
         num_rows: 27489
     })
     sampled_validation: Dataset({
         features: ['gem_id', 'gem_parent_id', 'source_language', 'target_language', 'source', 'target', 'references', 'input_ids', 'attention_mask', 'labels'],
         num_rows: 3000
     })
     sampled_test: Dataset({
         features: ['gem_id', 'gem_parent_id', 'source_language', 'target_language', 'source', 'target', 'references', 'input_ids', 'attention_mask', 'labels'],
       

In [12]:
combined_ds = [y['train'] for x,y in DS.items() if x in train_dialects]

combined_ds = concatenate_datasets(combined_ds).shuffle().select(range(500))

In [13]:
combined_ds

Dataset({
    features: ['gem_id', 'gem_parent_id', 'source_language', 'target_language', 'source', 'target', 'references', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [14]:


# for dialect in train_dialects:


#     torch.cuda.empty_cache()
#     model.add_adapter(dialect, config="seq_bn", overwrite_ok=True)
#     model.train_adapter(dialect)

#     training_args = Seq2SeqTrainingArguments(
#         learning_rate=1e-4,
#         num_train_epochs=2,
#         per_device_train_batch_size=8,
#         per_device_eval_batch_size=8,
#         logging_steps=200,
#         output_dir=f"/l/users/abdelrahman.sadallah/dialectal_gen/wiki_lingua-{dialect}",
#         overwrite_output_dir=True,
#         # The next line is important to ensure the dataset labels are properly passed to the model
#         remove_unused_columns=False,
#         predict_with_generate = True
#     )
#     trainer = Seq2SeqTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=DS[dialect]["train"] ,#  .select(range(10)),
#         eval_dataset=DS[dialect]["validation"].select(range(100)),
#         compute_metrics=compute_metrics_dia_wikilingua,
#         data_collator= DataCollatorForSeq2Seq(tokenizer, model=model,label_pad_token_id=-100,pad_to_multiple_of=8)
#     )
#     trainer.train()



#     model.save_adapter(f"/l/users/abdelrahman.sadallah/dialectal_gen/wiki_lingua-adapter-{dialect}", dialect, with_head=False)


#     DS[dialect].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


#     if dialect == "std-dia":
#         for test_dialect in test_dialects:
#             DS[test_dialect].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


            
#             eval_metrics = trainer.evaluate(DS[test_dialect]["validation"].select(range(100)))
#             eval_metrics['test_set'] = test_dialect
#             eval_metrics['train_set'] = dialect

#             results = pd.DataFrame(eval_metrics, index=[0])

#             print(results)
#             results.to_csv(f"train-{dialect}test-{test_dialect}-wikilingua.csv")
#     else:
#             test_dialect = dialect
#             DS[test_dialect].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
            
#             eval_metrics = trainer.evaluate(DS[test_dialect]["validation"].select(range(100)))
#             eval_metrics['test_set'] = test_dialect
#             eval_metrics['train_set'] = dialect

#             results = pd.DataFrame(eval_metrics, index=[0])

#             print(results)
#             results.to_csv(f"train-{dialect}test-{test_dialect}-wikilingua.csv")
        


In [15]:

   # fusuion

from adapters import T5AdapterModel
from adapters.composition import Fuse

model_name = 'google/flan-t5-base'
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# # init(model) 
model = T5AdapterModel.from_pretrained(model_name, )

for dialect in train_dialects:
    model.load_adapter(f"/l/users/abdelrahman.sadallah/dialectal_gen/wiki_lingua-adapter-{dialect}", with_head=False)



model.add_adapter_fusion(Fuse("std-dia", "aus", "hon", "nig", "col", "wel"))
model.set_active_adapters(Fuse("std-dia", "aus", "hon", "nig", "col", "wel"))
model.train_adapter_fusion(Fuse("std-dia", "aus", "hon", "nig", "col", "wel")) 


3################################## old that doesn't;t work ######################333
# model.set_active_adapters(train_dialects)
# model.add_adapter_fusion(train_dialects)
# # model.train_adapter_fusion(train_dialects) 

# # torch.cuda.empty_cache()
# model.add_adapter_fusion(train_dialects, "dynamic")
# model.train_adapter_fusion(train_dialects) 

3

In [16]:
model

T5AdapterModel(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5StackWithAdapters(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttentionWithAdapters(
              (SelfAttention): T5AttentionWithAdapters(
                (q): LoRALinearTorch(
                  in_features=768, out_features=768, bias=False
                  (loras): ModuleDict()
                )
                (k): LoRALinearTorch(
                  in_features=768, out_features=768, bias=False
                  (loras): ModuleDict()
                )
                (v): LoRALinearTorch(
                  in_features=768, out_features=768, bias=False
                  (loras): ModuleDict()
                )
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
                (prefix_tuning): Prefix

In [17]:



training_args = Seq2SeqTrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=200,
    output_dir=f"/l/users/abdelrahman.sadallah/dialectal_gen/wiki_lingua-fusion",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    predict_with_generate = True,
    # safe_serialization=False

)
# trainer = AdapterTrainer(
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=combined_ds ,#  .select(range(10)),
    eval_dataset=DS["std-dia"]["validation"].select(range(100)),
    compute_metrics=compute_metrics_dia_wikilingua,
    data_collator= DataCollatorForSeq2Seq(tokenizer, model=model,label_pad_token_id=-100,pad_to_multiple_of=8)
)
trainer.train()




    

Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bodasadallah2. Use `wandb login --relogin` to force relogin


Step,Training Loss
200,2.424800
400,2.269400
600,2.220000


TrainOutput(global_step=630, training_loss=2.2982847910078745, metrics={'train_runtime': 457.2652, 'train_samples_per_second': 10.935, 'train_steps_per_second': 1.378, 'total_flos': 7171526838730752.0, 'train_loss': 2.2982847910078745, 'epoch': 10.0})

In [18]:


model.save_adapter_fusion(f"/l/users/abdelrahman.sadallah/dialectal_gen/wiki_lingua-adapter-fusion", ["std-dia", "aus", "hon", "nig", "col", "wel"], with_head=False)


rr = []
for test_dialect in test_dialects:
    DS[test_dialect].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


    
    eval_metrics = trainer.evaluate(DS[test_dialect]["validation"].select(range(100)))
    eval_metrics['test_set'] = test_dialect
    eval_metrics['train_set'] = 'combined_dataset'

    rr.append(eval_metrics)




/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/utils.py:1156: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506caa17b50>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c983fa00>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c983f910>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c983fe20>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c81348e0>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c8134a00>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c81354b0>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c90bb9a0>
<class 'numpy.ndarray'>
hi
eval_preds <transformers.trainer_utils.EvalPrediction object at 0x1506c90b94e0>
<class 'numpy.ndarray'>


In [19]:
rr

[{'eval_loss': 1.8809133768081665,
  'eval_rouge1': 29.7665,
  'eval_rouge2': 12.3183,
  'eval_rougeL': 26.1264,
  'eval_rougeLsum': 28.8989,
  'eval_gen_len': 18.66,
  'eval_runtime': 15.2237,
  'eval_samples_per_second': 6.569,
  'eval_steps_per_second': 0.854,
  'epoch': 10.0,
  'test_set': 'std-dia',
  'train_set': 'combined_dataset'},
 {'eval_loss': 2.1812870502471924,
  'eval_rouge1': 31.3851,
  'eval_rouge2': 12.2592,
  'eval_rougeL': 27.4832,
  'eval_rougeLsum': 30.4854,
  'eval_gen_len': 18.95,
  'eval_runtime': 15.3533,
  'eval_samples_per_second': 6.513,
  'eval_steps_per_second': 0.847,
  'epoch': 10.0,
  'test_set': 'aus',
  'train_set': 'combined_dataset'},
 {'eval_loss': 2.7965574264526367,
  'eval_rouge1': 26.4739,
  'eval_rouge2': 7.997,
  'eval_rougeL': 22.9539,
  'eval_rougeLsum': 25.6177,
  'eval_gen_len': 18.46,
  'eval_runtime': 14.9752,
  'eval_samples_per_second': 6.678,
  'eval_steps_per_second': 0.868,
  'epoch': 10.0,
  'test_set': 'hon',
  'train_set': 'comb

In [20]:
results = pd.DataFrame(rr)
# results = pd.DataFrame.from_records(rr)
print(results)
results.to_csv(f"train-combined-test-{test_dialect}-wikilingua_fusion.csv")


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0   1.880913      29.7665      12.3183      26.1264         28.8989   
1   2.181287      31.3851      12.2592      27.4832         30.4854   
2   2.796557      26.4739       7.9970      22.9539         25.6177   
3   2.368487      30.0533      10.4181      26.5662         29.0674   
4   2.122571      29.3830      10.5500      24.9296         28.2667   
5   2.481393      28.6504       8.4752      24.8358         27.6161   
6   2.023733      28.1166      10.1366      23.5628         26.9295   
7   2.342084      27.0826       9.0722      23.5643         26.3302   
8   2.411781      25.7012       7.2278      21.2734         24.4264   

   eval_gen_len  eval_runtime  eval_samples_per_second  eval_steps_per_second  \
0         18.66       15.2237                    6.569                  0.854   
1         18.95       15.3533                    6.513                  0.847   
2         18.46       14.9752                 

In [21]:
model

T5AdapterModel(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5StackWithAdapters(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttentionWithAdapters(
              (SelfAttention): T5AttentionWithAdapters(
                (q): LoRALinearTorch(
                  in_features=768, out_features=768, bias=False
                  (loras): ModuleDict()
                )
                (k): LoRALinearTorch(
                  in_features=768, out_features=768, bias=False
                  (loras): ModuleDict()
                )
                (v): LoRALinearTorch(
                  in_features=768, out_features=768, bias=False
                  (loras): ModuleDict()
                )
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
                (prefix_tuning): Prefix